In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import re
import spacy
import nltk
from nltk import tokenize
from collections import Counter
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

# Creating Dataset for Entity Recognition

In [2]:
df = pd.read_csv('C:/Users/wongzn/Desktop/df_dec.csv')

new_df = df
new_df.reset_index(inplace=True)
new_df.drop(columns=['index'],inplace=True)

In [3]:
def preprocess_content(column):
    column = column.apply(lambda x: re.sub(r'SINGAPORE -', '', x))
    #column = column.apply(lambda x: re.sub(r'the\b', '', x,flags=re.IGNORECASE))
    column = column.apply(lambda x: re.sub(r'SINGAPORE –', ' ', x, flags=re.IGNORECASE))
    column = column.apply(lambda x: re.sub(r'SINGAPORE-', ' ', x, flags=re.IGNORECASE))
    column = column.apply(lambda x: re.sub(r'\([^)]*\)', '',x)) #remove (anything in brackets)
    #column = column.apply(lambda x: re.sub(r'\d{2}[/-]\w{4}[/-]\w{3,4}', '',x)) #remove(xx-year-old)
    #column = column.apply(lambda x: re.sub(r'straits times','',x, flags=re.IGNORECASE))
    #column = column.apply(lambda x: re.sub(r'per cent','',x, flags=re.IGNORECASE))
    #column = column.apply(lambda x: re.sub(r'cpl kok yuen chin','',x, flags=re.IGNORECASE))
    #column = column.apply(lambda x: re.sub(r'cpl kok','',x, flags=re.IGNORECASE))
    #column = column.apply(lambda x: re.sub(r'st photo','',x, flags=re.IGNORECASE))
    column = column.apply(lambda x: re.sub(r'facebook post','',x, flags=re.IGNORECASE))
    #column = column.apply(lambda x: re.sub(r'years ago','',x, flags=re.IGNORECASE))
    #column = column.apply(lambda x: re.sub(r'[^a-zA-z\s]', ' ', x)) #removing digits and special characters (punctuations)
    #column = column.apply(lambda x: x.lower()) #convert to lowercase
    column = column.apply(lambda x: re.sub('\n','',x))
    return column

new_df['Article Content'] = preprocess_content(new_df['Article Content'])

In [4]:
sentences = []
count_sent = []
for i in range(len(new_df)):
    #Split into sentences
    sentences.append([x for x in tokenize.sent_tokenize(new_df['Article Content'][i])])
    #Counting the number of sentences in each article
    count_sent.append(len(tokenize.sent_tokenize(new_df['Article Content'][i])))

#Flatten list of sentences
new =  [item for sublist in sentences for item in sublist]

In [5]:
new_df = pd.concat([pd.Series(new)],axis=1)
new_df.rename(columns={0:'Sent'},inplace=True)

In [6]:
rm = 'ST PHOTO'
new2 = [item for item in new if rm not in item] #Removing sentences with ST Photo

df_sent = pd.DataFrame(new2, columns=['Sent'])
df_sent.drop_duplicates(subset=['Sent'], inplace=True)

In [7]:
df_sent.to_csv('C:/Users/wongzn/Desktop/df_sent.csv')

## Identifying Entities Using spaCy

In [8]:
ner = df_sent.Sent.apply(nlp)

In [9]:
word = []
pos = []
iob = []
entity = []
sent_num = []
count = 0

for ent in range(len(ner)):
    try:
        count +=1
        for i in ner[ent]:
            word.append(i.text)
            iob.append(i.ent_iob_)
            entity.append(i.ent_type_)
            pos.append(i.pos_)
            sent_num.append("Sentence " + str(count))
    except:
        continue

In [10]:
ent_df = pd.DataFrame(list(zip(sent_num,word,pos,iob,entity)), columns=['Sentence Number','Word', 'POS', 'IOB','Entity'])

In [11]:
ent_df.replace('', np.nan, inplace=True)
ent_df.isnull().sum()

Sentence Number          0
Word                     0
POS                      0
IOB                      0
Entity             1438105
dtype: int64

In [12]:
ent_df.fillna('O',inplace=True)

In [13]:
ent_df.Entity.value_counts()

O              1438105
ORG              70821
DATE             58748
PERSON           39809
CARDINAL         22288
GPE              15682
MONEY            12406
FAC               8251
TIME              4598
NORP              4284
LOC               4258
ORDINAL           3031
QUANTITY          2085
EVENT             1939
LAW               1925
WORK_OF_ART       1567
PRODUCT            943
LANGUAGE           195
PERCENT             38
Name: Entity, dtype: int64

In [14]:
ent_df['Final'] = ent_df['IOB']+'-'+ ent_df['Entity']
ent_df['Final'].replace('O-O','O',inplace=True)

In [15]:
focus = ['ORG','LOC','PERSON','GPE','FAC'] #Only working on these entities
pol_df = ent_df[ent_df['Entity'].isin(focus)]
pol_df.Entity.value_counts()

ORG       70821
PERSON    39809
GPE       15682
FAC        8251
LOC        4258
Name: Entity, dtype: int64

In [16]:
#pol_df.to_csv('C:/Users/wongzn/Desktop/Entity_Data_subset.csv')

# Exploration of Entities 

### Getting Frequency Of Each Entity Type

In [17]:
ner = [nlp(s) for s in df_sent.Sent]

In [18]:
net = [[w.ent_type_ for w in s if w.ent_type_] for s in ner]

In [19]:
net = [s if s else ['None'] for s in net]

In [20]:
ners = [ne for n in net for ne in n]

In [21]:
from collections import Counter

ner_cnt = Counter(ners)
ner_cnt.most_common(10)

[('ORG', 71741),
 ('DATE', 59585),
 ('PERSON', 40534),
 ('CARDINAL', 22577),
 ('GPE', 15873),
 ('None', 14981),
 ('MONEY', 12499),
 ('FAC', 8327),
 ('TIME', 4666),
 ('NORP', 4365)]

In [22]:
NE = pd.DataFrame(ners, columns=['NE'])

In [23]:
NE_count = pd.DataFrame(NE.NE.value_counts())
NE_count = NE_count.rename_axis('Entity').reset_index()
NE_count.rename(columns={'NE':'Freq'},inplace=True)
NE_count['Perc'] = round((NE_count.Freq/sum(NE_count.Freq) * 100),2).astype(str) +'%'

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(22,5))
plt.bar(NE_count['Entity'],NE_count['Freq'])
plt.xticks(rotation=75)
for i in range(len(NE_count)):
    plt.text(NE_count.Entity[i],NE_count.Freq[i], NE_count.Perc[i], color='black', ha="center",size=12)
plt.show()

<Figure size 2200x500 with 1 Axes>

### Getting Top Frequency in Each Entity Type

In [25]:
k=[]
filt=['PERSON']

for ent in ner:
    for i in ent.ents:
        if i.label_ in filt:
            k.append(i.text)
            
k = pd.DataFrame(k,columns=['PERSON'])
k = pd.DataFrame(k.PERSON.value_counts()).rename(columns={'PERSON':'Freq'})
k = k.rename_axis('Entity').reset_index()
k.head(10)

,Entity,Freq
0,Tan,325
1,Lee,261
2,PM Lee,198
3,Ong,179
4,Lee Hsien Loong,147
5,Halimah Yacob,141
6,Lim,136
7,Mr Heng,132
8,Ang Mo Kio,116
9,Chan,114


# Named Entity Recognition

In [26]:
df1 = pd.read_csv('C:/Users/wongzn/Desktop/Entity_Data.csv')
df2 = pd.read_csv('C:/Users/wongzn/Desktop/Entity_Data_Edit.csv') #Edited the entities manually
#df2.drop(columns=['Final'],inplace=True)

In [27]:
df = df1.merge(df2, on=['Unnamed: 0'], how='left')[:80000]
df.IOB_y.fillna(df.IOB_x, inplace=True)
df.Entity_y.fillna(df.Entity_x, inplace=True)
df.drop(columns=['Unnamed: 0','IOB_x','Entity_x','Sentence Number_y','Word_y','POS_y'], inplace=True)
df.rename(columns={'Sentence Number_x':"Sentence Number",'Word_x': 'Word','POS_x':'POS','IOB_y':'IOB','Entity_y':'Entity'}, inplace=True)

In [28]:
#df.to_csv("C:/Users/wongzn/Desktop/Entity_Data_Edit.csv")

In [29]:
df.Entity.value_counts()

O              68073
DATE            2481
ORG             2352
PERSON          1911
CARDINAL        1303
LOC              679
MONEY            605
FAC              478
GPE              382
TITLE            373
TIME             287
QUANTITY         205
NORP             161
EVENT            160
PERCENT          158
LAW              135
ORDINAL          125
PRODUCT           80
WORK_OF_ART       26
LANGUAGE          26
Name: Entity, dtype: int64

In [30]:
df['Tag'] = np.where(df.IOB.isin(['O'])|df.Entity.isin(['O']), 'O', df.IOB.str.strip() + '-' + df.Entity )
#df.drop(columns=['IOB','Entity'],inplace=True)

In [31]:
df.Word.fillna('null',inplace=True)
df.isna().sum()

Sentence Number    0
Word               0
POS                0
IOB                0
Entity             0
Tag                0
dtype: int64

In [32]:
df.groupby('Tag').size().reset_index(name='counts')
X = df.drop('Tag', axis=1)

In [33]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
X.shape

(80000, 12401)

In [34]:
y = df.Tag.values

In [35]:
classes = np.unique(y)
classes = classes.tolist()

In [36]:
X.shape, y.shape

((80000, 12401), (80000,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)

In [39]:
new_classes = classes.copy()
new_classes.pop() #Remove O

'O'

In [40]:
new_classes = ['B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC']

# Modelling

In [ ]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train,classes)
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

In [24]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

     B-TITLE       1.00      1.00      1.00        61
     I-TITLE       1.00      1.00      1.00        68
    B-PERSON       1.00      1.00      1.00       333
    I-PERSON       1.00      1.00      1.00       317
       B-ORG       1.00      1.00      1.00       346
       I-ORG       1.00      1.00      1.00       447
       B-FAC       1.00      1.00      1.00        57
       I-FAC       1.00      1.00      1.00        98
       B-LOC       1.00      1.00      1.00       100
       I-LOC       1.00      1.00      1.00       109

   micro avg       1.00      1.00      1.00      1936
   macro avg       1.00      1.00      1.00      1936
weighted avg       1.00      1.00      1.00      1936



In [21]:
from sklearn import model_selection, naive_bayes, svm
SVM = svm.SVC(C=1.6, kernel='linear',random_state=1)
SVM.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy
print(classification_report(y_pred=predictions_SVM, y_true=y_test, labels=new_classes))

C:\Users\wongzn\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\wongzn\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


               precision    recall  f1-score   support

   B-CARDINAL       0.52      0.68      0.59       414
       B-DATE       0.66      0.56      0.60       454
      B-EVENT       0.17      0.07      0.10        15
        B-FAC       0.63      0.38      0.48        50
        B-GPE       0.60      0.74      0.67       117
   B-LANGUAGE       0.75      0.86      0.80         7
        B-LAW       0.25      0.06      0.10        17
        B-LOC       0.64      0.62      0.63        92
      B-MONEY       0.29      0.10      0.15        99
       B-NORP       0.95      0.65      0.77        55
    B-ORDINAL       0.87      0.91      0.89        45
        B-ORG       0.56      0.35      0.43       317
    B-PERCENT       0.00      0.00      0.00         0
     B-PERSON       0.50      0.38      0.43       267
    B-PRODUCT       0.68      0.62      0.65        24
   B-QUANTITY       0.40      0.08      0.13        26
       B-TIME       0.43      0.25      0.32        24
      B-T

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn=KNeighborsClassifier(n_neighbors=37)
knn.fit(X_train,y_train)
knn_predictions = knn.predict(X_test)
print(classification_report(y_pred=knn_predictions, y_true=y_test, labels=new_classes))

In [ ]:
from sklearn import tree
tree = tree.DecisionTreeClassifier(random_state = 21)
tree.fit(X_train,y_train)
tree_prediction = tree.predict(X_test)
print(classification_report(y_pred=tree_prediction, y_true=y_test, labels=new_classes))

In [52]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(multi_class='multinomial',solver='newton-cg')
LR.fit(X_train,y_train)
pred_LR = LR.predict(X_test)
print(classification_report(y_pred=pred_LR, y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       1.00      0.11      0.20         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.42      0.72      0.53        69
       B-gpe       0.77      0.60      0.67       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.48      0.51      0.50        63
       B-per       0.71      0.49      0.58        41
       B-tim       1.00      0.79      0.88        52
       I-art       0.00      0.00      0.00        10
       I-eve       0.00      0.00      0.00         3
       I-geo       0.00      0.00      0.00        11
       I-gpe       1.00      0.17      0.29         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.50      0.34      0.41        47
       I-per       0.54      0.58      0.56        66
       I-tim       1.00      0.25      0.40         4

   micro avg       0.59      0.54      0.56       487
   macro avg       0.46   

C:\Users\wongzn\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\wongzn\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [53]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(random_state=1)
nn.fit(X_train,y_train)
nn_prediction = nn.predict(X_test)
print(classification_report(y_pred=nn_prediction, y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.60      0.33      0.43         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.52      0.57      0.54        69
       B-gpe       0.72      0.75      0.73       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.49      0.52      0.51        63
       B-per       0.42      0.49      0.45        41
       B-tim       0.92      0.90      0.91        52
       I-art       0.43      0.30      0.35        10
       I-eve       1.00      0.33      0.50         3
       I-geo       0.57      0.36      0.44        11
       I-gpe       0.75      0.50      0.60         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.49      0.43      0.45        47
       I-per       0.62      0.48      0.54        66
       I-tim       0.25      0.25      0.25         4

   micro avg       0.60      0.58      0.59       487
   macro avg       0.49   

C:\Users\wongzn\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\wongzn\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# CRF Model For NER

In [41]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [42]:
df1 = pd.read_csv('C:/Users/wongzn/Desktop/Entity_Data.csv')
df2 = pd.read_csv('C:/Users/wongzn/Desktop/Entity_Data_Edit.csv')
#df2.drop(columns=['Final'],inplace=True)
df = df1.merge(df2, on=['Unnamed: 0'], how='left')[:80000]
df.IOB_y.fillna(df.IOB_x, inplace=True)
df.Entity_y.fillna(df.Entity_x, inplace=True)
df.drop(columns=['Unnamed: 0','IOB_x','Entity_x','Sentence Number_y','Word_y','POS_y'], inplace=True)
df.rename(columns={'Sentence Number_x':"Sentence Number",'Word_x': 'Word','POS_x':'POS','IOB_y':'IOB','Entity_y':'Entity'}, inplace=True)
df['Tag'] = np.where(df.IOB.isin(['O'])|df.Entity.isin(['O']), 'O', df.IOB.str.strip() + '-' + df.Entity )
df.drop(columns=['IOB','Entity'],inplace=True)

In [43]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence Number').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [44]:
getter = SentenceGetter(df)
sent = getter.get_next()
#print(sent)

In [45]:
sentences = getter.sentences

In [46]:
def word2features(sent, i):
    word = str(sent[i][0])
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = str(sent[i-1][0])
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = str(sent[i+1][0])
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [47]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [49]:
import time

In [50]:
st = time.time()
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=True)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [51]:
time.time()-st

28.678688526153564

In [52]:
new_classes = ['B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC']

In [53]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

0.7262327040614024

In [54]:
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

     B-TITLE       0.73      0.47      0.57        64
     I-TITLE       0.49      0.29      0.36        73
    B-PERSON       0.77      0.83      0.80       350
    I-PERSON       0.80      0.87      0.83       350
       B-ORG       0.68      0.65      0.67       330
       I-ORG       0.68      0.73      0.71       393
       B-FAC       0.81      0.53      0.64        55
       I-FAC       0.88      0.50      0.64       100
       B-LOC       0.81      0.81      0.81       118
       I-LOC       0.77      0.80      0.79       108

   micro avg       0.74      0.72      0.73      1941
   macro avg       0.74      0.65      0.68      1941
weighted avg       0.74      0.72      0.73      1941



# CRF Gridsearch

In [55]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

crf = sklearn_crfsuite.CRF(algorithm='lbfgs', max_iterations=100,all_possible_transitions=True)
params_space = {'c1': scipy.stats.expon(scale=0.5),'c2': scipy.stats.expon(scale=0.05),}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=new_classes)

# search
st = time.time()
rs = RandomizedSearchCV(crf, params_space, cv=3, verbose=1,n_jobs=-1, n_iter=50, scoring=f1_scorer)
rs.fit(X_train, y_train)
print(time.time()-st)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 42.7min finished


2649.630345582962


In [56]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.006270641912923742, 'c2': 0.09073265475048321}
best CV score: 0.7489902505576554
model size: 1.11M


In [57]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=new_classes))

              precision    recall  f1-score   support

     B-TITLE       0.77      0.47      0.58        64
     I-TITLE       0.58      0.29      0.39        73
    B-PERSON       0.78      0.81      0.79       350
    I-PERSON       0.81      0.85      0.83       350
       B-ORG       0.68      0.65      0.67       330
       I-ORG       0.68      0.72      0.70       393
       B-FAC       0.79      0.56      0.66        55
       I-FAC       0.92      0.54      0.68       100
       B-LOC       0.81      0.81      0.81       118
       I-LOC       0.80      0.80      0.80       108

   micro avg       0.75      0.72      0.73      1941
   macro avg       0.76      0.65      0.69      1941
weighted avg       0.75      0.72      0.73      1941



# Feeding in New Data (Test Dataset)

In [58]:
df_test = pd.read_csv('C:/Users/wongzn/Desktop/CRF_test_data.csv')[13:18701]
df_test.drop(columns=['Unnamed: 0'],inplace=True)

In [59]:
df_test.tail()

,Sentence Number,Word,POS,IOB,Entity_Original
18696,Sentence 22093,years,NOUN,I,DATE
18697,Sentence 22093,and,CCONJ,O,O
18698,Sentence 22093,fined,VERB,O,O
18699,Sentence 22093,.,PUNCT,O,O
18700,Sentence 22094,,SPACE,O,O


In [ ]:
#df_test = df1_test.merge(df2_test, on=['Unnamed: 0'], how='left')[557804:611740]
#df_test.IOB_y.fillna(df_test.IOB_x, inplace=True)
#df_test.Entity_y.fillna(df_test.Entity_x, inplace=True)
#df_test.drop(columns=['Unnamed: 0','IOB_x','Entity_x','Sentence Number_y','Word_y','POS_y'], inplace=True)
#df_test.rename(columns={'Sentence Number_x':"Sentence Number",'Word_x': 'Word','POS_x':'POS','IOB_y':'IOB','Entity_y':'Entity_Original'}, inplace=True)

In [60]:
df_test['Tag'] = np.where(df_test.IOB.isin(['O'])|df_test.Entity_Original.isin(['O']), 'O', df_test.IOB.str.strip() + '-' + df_test.Entity_Original )
df_test.drop(columns=['IOB'],inplace=True)

In [61]:
getter_test = SentenceGetter(df_test)

In [62]:
sentences_test = getter_test.sentences

In [63]:
X_train = [sent2features(s) for s in sentences]
X_test = [sent2features(s) for s in sentences_test]
Y_train = [sent2labels(s) for s in sentences]
Y_test = [sent2labels(s) for s in sentences_test]

In [64]:
st = time.time()
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=True)
crf.fit(X_train, Y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [65]:
time.time()-st

132.8361439704895

In [66]:
y_pred = crf.predict(X_test)
#metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

In [67]:
flat_list_true = [item for sublist in Y_test for item in sublist]

In [68]:
flat_list_value = df_test.Word

In [69]:
flat_list_pred = [item for sublist in y_pred for item in sublist]

In [70]:
test_df = pd.DataFrame()

In [71]:
test_df['Word'] = [x for x in flat_list_value]
test_df['Original'] = flat_list_true
test_df['Pred'] = flat_list_pred

In [72]:
test_df.reset_index(inplace=True)

In [73]:
test_df.drop(columns=['index'],inplace=True)

In [75]:
test_df.head()

,Word,Original,Pred
0,Singaporeans,B-NORP,B-NORP
1,should,O,O
2,defer,O,O
3,non,O,O
4,-,O,O


In [ ]:
#test_df.to_csv("C:/Users/wongzn/Desktop/CRF_res.csv")

# Getting Entities from CRF Model Prediction

In [76]:
location = []
for i in range(len(test_df)):
    if test_df.Pred.iloc[i] == 'B-LOC':
        word = test_df.Word.iloc[i]
        count = i
        while test_df.Pred.iloc[count+1] == 'I-LOC':
            word = word + ' ' + test_df.Word.iloc[count+1]
            count +=1
        else: 
            word = word
        location.append(word)

In [77]:
loc_df = pd.DataFrame(location,columns=['Loc'])
loc_df = pd.DataFrame(loc_df.Loc.value_counts()).rename(columns={'Loc':'Freq'})
loc_df = loc_df.rename_axis('Entity').reset_index()

In [78]:
loc_df.head(10)

,Entity,Freq
0,Asia Chapter,4
1,Seletar Airport,2
2,Subang,2
3,Pedra Branca,2
4,Causeway Bay,2
5,Serangoon Road,1
6,Rakhine,1
7,Telok Kurau,1
8,Punggol,1
9,Choa Chu Kang,1


# CRF Results Exploration

In [79]:
from collections import Counter

label_from = ['O','B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC']
label_to = ['O','B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC']

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        a =['O','B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC']
        if label_from in a and label_to in a:
            print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-FAC  -> I-FAC   7.052096
B-ORG  -> I-ORG   6.022573
I-ORG  -> I-ORG   5.918308
I-FAC  -> I-FAC   5.634691
B-PERSON -> I-PERSON 5.481820
B-TITLE -> I-TITLE 5.377037
I-PERSON -> I-PERSON 5.291522

Top unlikely transitions:
O      -> I-FAC   -1.878616
I-TITLE -> I-ORG   -1.972712
O      -> I-LOC   -2.061899
O      -> I-PERSON -2.489468
O      -> I-ORG   -2.588420
O      -> I-TITLE -3.209935


In [80]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        a =['B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC']
        if label in a:
            print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-100:])

Top positive:
4.227195 B-ORG    word.lower():parliament
4.071964 B-ORG    word.lower():wang
3.899756 B-TITLE  word.lower():president
3.612791 B-PERSON -1:word.lower():judge
3.485618 B-ORG    word[-3:]:ram
3.391730 B-ORG    word.lower():twitter

Top negative:
-1.372623 I-LOC    -1:word.lower():west
-1.442770 B-PERSON -1:word.lower():-
-1.833212 I-LOC    -1:word.lower():kio


In [81]:
import eli5

eli5.show_weights(crf, top=10)

Using TensorFlow backend.


From \ To,O,B-CARDINAL,I-CARDINAL,B-DATE,I-DATE,B-EVENT,I-EVENT,B-FAC,I-FAC,B-GPE,I-GPE,B-LANGUAGE,I-LANGUAGE,B-LAW,I-LAW,B-LOC,I-LOC,B-MONEY,I-MONEY,B-NORP,I-NORP,B-ORDINAL,B-ORG,I-ORG,B-PERCENT,I-PERCENT,B-PERSON,I-PERSON,B-PRODUCT,I-PRODUCT,B-QUANTITY,I-QUANTITY,B-TIME,I-TIME,B-TITLE,I-TITLE,B-WORK_OF_ART,I-WORK_OF_ART
O,3.774,2.28,-3.191,1.979,-4.369,0.347,-2.114,0.867,-1.879,0.698,-1.216,0.017,-1.186,1.202,-3.101,1.04,-2.062,2.416,-4.514,0.796,-1.5,0.944,1.478,-2.588,1.179,-2.073,1.257,-2.489,0.109,-1.972,1.294,-2.324,0.981,-3.193,0.405,-3.21,0.407,-1.997
B-CARDINAL,1.174,0.0,5.097,0.205,-2.732,0.043,0.0,-0.089,-0.53,-0.029,0.0,0.0,0.0,0.0,0.0,-0.359,-1.009,-1.002,-1.616,1.176,0.0,-0.119,1.257,-0.761,0.0,-1.069,0.0,-0.688,0.925,-0.205,0.0,-1.654,-0.386,-1.944,-0.37,-0.227,0.0,-0.065
I-CARDINAL,0.802,-0.993,5.356,-1.224,-1.673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.592,-0.794,0.072,0.0,0.0,-0.244,0.0,-0.413,-1.114,0.0,0.0,0.657,0.0,-0.049,-0.349,-0.197,-0.795,0.0,0.0,0.0,0.0
B-DATE,0.338,-1.073,-0.925,-0.542,5.228,0.159,0.0,-0.061,0.0,-0.378,0.0,0.0,0.0,0.0,-0.274,-1.172,-0.624,0.507,-1.181,-0.375,0.0,-0.956,0.004,-0.504,0.0,-0.15,-0.67,-0.221,-0.652,-0.315,-0.153,-0.36,1.473,-1.457,-0.035,-0.09,0.0,0.0
I-DATE,-0.641,-1.259,-1.496,-0.129,5.332,-1.177,-0.0,0.0,-0.004,0.0,0.0,0.0,0.0,0.0,-0.306,0.0,-0.109,-0.237,-0.682,0.0,0.0,-0.038,-0.621,-0.726,0.0,-0.681,-0.743,0.0,0.61,0.0,-0.075,-0.043,-0.155,-0.35,-0.293,-0.352,0.0,0.0
B-EVENT,-0.571,0.0,0.0,0.0,-0.091,0.0,6.057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.061,-0.175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-EVENT,-0.494,0.0,0.0,0.0,-0.457,0.0,6.814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.016,0.0,0.0,0.0,0.0,0.0,0.0,-0.223,0.0,0.0,-0.214,-0.116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-FAC,0.053,0.0,0.0,0.0,-0.139,0.0,0.0,0.0,7.052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.396,0.0,0.0,0.0,0.0,0.0,-0.279,-0.044,0.0,0.0,-0.078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-FAC,-0.312,-0.335,0.0,0.557,-0.16,0.0,0.0,0.936,5.635,-0.061,-0.0,0.0,0.0,0.0,0.0,0.0,-0.695,0.0,0.0,0.0,0.0,0.0,-0.182,-0.736,0.0,0.0,-0.709,-0.444,0.0,0.0,0.0,0.0,0.143,0.0,-0.132,0.0,0.0,0.0
B-GPE,0.451,-0.328,0.0,0.379,-0.468,0.0,-0.089,0.0,-0.789,0.0,5.195,0.0,0.0,0.0,-0.189,0.0,-0.416,0.0,0.0,0.0,0.0,0.0,0.618,-1.218,0.0,0.0,-0.5,-0.255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(X_train, Y_train);
eli5.show_weights(crf, top=5, show=['transition_features'])

From \ To,O,B-CARDINAL,I-CARDINAL,B-DATE,I-DATE,B-EVENT,I-EVENT,B-FAC,I-FAC,B-GPE,I-GPE,B-LANGUAGE,I-LANGUAGE,B-LAW,I-LAW,B-LOC,I-LOC,B-MONEY,I-MONEY,B-NORP,I-NORP,B-ORDINAL,B-ORG,I-ORG,B-PERCENT,I-PERCENT,B-PERSON,I-PERSON,B-PRODUCT,I-PRODUCT,B-QUANTITY,I-QUANTITY,B-TIME,I-TIME,B-TITLE,I-TITLE,B-WORK_OF_ART,I-WORK_OF_ART
O,3.774,2.28,-3.191,1.979,-4.369,0.347,-2.114,0.867,-1.879,0.698,-1.216,0.017,-1.186,1.202,-3.101,1.04,-2.062,2.416,-4.514,0.796,-1.5,0.944,1.478,-2.588,1.179,-2.073,1.257,-2.489,0.109,-1.972,1.294,-2.324,0.981,-3.193,0.405,-3.21,0.407,-1.997
B-CARDINAL,1.174,0.0,5.097,0.205,-2.732,0.043,0.0,-0.089,-0.53,-0.029,0.0,0.0,0.0,0.0,0.0,-0.359,-1.009,-1.002,-1.616,1.176,0.0,-0.119,1.257,-0.761,0.0,-1.069,0.0,-0.688,0.925,-0.205,0.0,-1.654,-0.386,-1.944,-0.37,-0.227,0.0,-0.065
I-CARDINAL,0.802,-0.993,5.356,-1.224,-1.673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.592,-0.794,0.072,0.0,0.0,-0.244,0.0,-0.413,-1.114,0.0,0.0,0.657,0.0,-0.049,-0.349,-0.197,-0.795,0.0,0.0,0.0,0.0
B-DATE,0.338,-1.073,-0.925,-0.542,5.228,0.159,0.0,-0.061,0.0,-0.378,0.0,0.0,0.0,0.0,-0.274,-1.172,-0.624,0.507,-1.181,-0.375,0.0,-0.956,0.004,-0.504,0.0,-0.15,-0.67,-0.221,-0.652,-0.315,-0.153,-0.36,1.473,-1.457,-0.035,-0.09,0.0,0.0
I-DATE,-0.641,-1.259,-1.496,-0.129,5.332,-1.177,-0.0,0.0,-0.004,0.0,0.0,0.0,0.0,0.0,-0.306,0.0,-0.109,-0.237,-0.682,0.0,0.0,-0.038,-0.621,-0.726,0.0,-0.681,-0.743,0.0,0.61,0.0,-0.075,-0.043,-0.155,-0.35,-0.293,-0.352,0.0,0.0
B-EVENT,-0.571,0.0,0.0,0.0,-0.091,0.0,6.057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.061,-0.175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-EVENT,-0.494,0.0,0.0,0.0,-0.457,0.0,6.814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.016,0.0,0.0,0.0,0.0,0.0,0.0,-0.223,0.0,0.0,-0.214,-0.116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-FAC,0.053,0.0,0.0,0.0,-0.139,0.0,0.0,0.0,7.052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.396,0.0,0.0,0.0,0.0,0.0,-0.279,-0.044,0.0,0.0,-0.078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-FAC,-0.312,-0.335,0.0,0.557,-0.16,0.0,0.0,0.936,5.635,-0.061,-0.0,0.0,0.0,0.0,0.0,0.0,-0.695,0.0,0.0,0.0,0.0,0.0,-0.182,-0.736,0.0,0.0,-0.709,-0.444,0.0,0.0,0.0,0.0,0.143,0.0,-0.132,0.0,0.0,0.0
B-GPE,0.451,-0.328,0.0,0.379,-0.468,0.0,-0.089,0.0,-0.789,0.0,5.195,0.0,0.0,0.0,-0.189,0.0,-0.416,0.0,0.0,0.0,0.0,0.0,0.618,-1.218,0.0,0.0,-0.5,-0.255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
eli5.show_weights(crf, top=10, targets=['O','B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC'])

From \ To,O,B-TITLE,I-TITLE,B-PERSON,I-PERSON,B-ORG,I-ORG,B-FAC,I-FAC,B-LOC,I-LOC
O,3.774,0.405,-3.21,1.257,-2.489,1.478,-2.588,0.867,-1.879,1.04,-2.062
B-TITLE,-0.768,0.218,5.377,1.78,0.358,-0.359,-1.228,0.0,-0.026,0.0,-0.08
I-TITLE,-1.036,0.0,4.752,2.156,-1.252,0.014,-1.973,0.0,-0.098,-0.012,-0.18
B-PERSON,0.203,-0.118,-0.222,-1.707,5.482,-0.603,-0.789,0.0,-0.846,0.0,-0.786
I-PERSON,0.272,-0.048,-0.12,-1.404,5.292,-0.219,0.184,0.0,-0.595,0.0,-0.679
B-ORG,0.423,0.0,0.002,-0.287,-1.373,-0.925,6.023,-0.145,0.564,-0.088,0.0
I-ORG,0.027,1.401,-1.064,0.188,-0.26,-0.534,5.918,-0.2,-0.852,-0.079,-0.969
B-FAC,0.053,0.0,0.0,-0.078,0.0,-0.279,-0.044,0.0,7.052,0.0,-0.396
I-FAC,-0.312,-0.132,0.0,-0.709,-0.444,-0.182,-0.736,0.936,5.635,0.0,-0.695
B-LOC,-0.109,0.0,-0.014,-1.191,-1.062,-1.063,-0.984,0.0,-0.912,1.495,5.275


In [84]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'],targets=['B-TITLE','I-TITLE','B-PERSON','I-PERSON','B-ORG','I-ORG','B-FAC','I-FAC','B-LOC','I-LOC','B-EVENT','I-EVENT'])

Weight?,Feature
+1.202,word.isupper()
-0.081,word.istitle()
-0.396,word.isdigit()
Weight?,Feature
+0.152,word.istitle()
-0.235,word.isupper()
Weight?,Feature
+2.607,word.istitle()
+0.251,word.isupper()
-0.137,word.isdigit()
